In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate

In [ ]:
input_shape = (224, 224, 3)
classes = 1000

In [ ]:
def inception_block(x, filters, stage):
  """
  arg
    - filters:
      example => [64, (96, 128), (16, 32), 32]
  """
  # 1x1 conv
  path_a = Conv2D(filters[0], 1, padding='same', activation='relu',
                  name=stage+'_a_conv')(x)

  # 1x1 conv 3x3 conv
  path_b = Conv2D(filters[1][0], 1, padding='same', activation='relu',
                  name=stage+'_b_conv_1')(x)
  path_b = Conv2D(filters[1][1], 3, padding='same', activation='relu',
                  name=stage+'_b_conv_2')(path_b)

  # 1x1 conv 5x5 conv
  path_c = Conv2D(filters[2][0], 1, padding='same', activation='relu',
                  name=stage+'_c_conv_1')(x)
  path_c = Conv2D(filters[2][1], 5, padding='same', activation='relu',
                  name=stage+'_c_conv_2')(path_c)

  # 3x3 pooing 1x1 conv
  path_d = MaxPooling2D(3, 1, padding='same', name=stage+'_d_pool')(x)
  path_d = Conv2D(filters[3], 1, padding='same', activation='relu',
                  name=stage+'_d_conv')(path_d)

  return Concatenate(axis=-1, name=stage+'_concat')([path_a, path_b, path_c, path_d])

In [ ]:
def auxiliary_block(x, name):
  x = AveragePooling2D(5, 3)(x)
  x = Conv2D(128, 1, padding='same', activation='relu')(x)
  x = Flatten()(x)
  x = Dense(256, 'relu')(x)
  return Dense(classes, 'softmax', name=name)(x)

In [ ]:
input = keras.Input(shape=input_shape)

# stage-1
x = Conv2D(64, 7, 2, 'same', activation='relu', name='stage1_conv')(input) # (112, 112, 64)
x = MaxPooling2D(3, 2, 'same', name='stage1_pool')(x) # (56, 56, 64)
x = BatchNormalization(name='stage1_bn')(x) # (56, 56, 64)

# stage-2
x = Conv2D(64, 1, 1, 'same', activation='relu', name='stage2_conv1')(x) # (56, 56, 64)
x = Conv2D(192, 3, 1, 'same', activation='relu', name='stage2_conv2')(x) # (56, 56, 192)
x = BatchNormalization(name='stage2_bn')(x) # (56, 56, 192)
x = MaxPooling2D(3, 2, 'same', name='stage2_pool')(x) # (28, 28, 192)

# stage-3
x = inception_block(x, [64, (96,128), (16,32), 32], 'stage3-1') # =>(28, 28, 256)
x = inception_block(x, [128, (128,192), (32,96), 64], 'stage3-2') # =>(28, 28, 480)
x = MaxPooling2D(3, 2, 'same', name='stage3_pool')(x) # => (14, 14, 480)

# stage-4
x = inception_block(x, [192, (96,208), (16,48), 64], 'stage4-1') # (14, 14, 512)
output_aux1 = auxiliary_block(x, 'aux1') # (1000, )
x = inception_block(x, [160, (112,224), (24,64), 64], 'stage4-2') # (14, 14, 512)
x = inception_block(x, [128, (128,256), (24,64), 64], 'stage4-3') # (14, 14, 512)
x = inception_block(x, [112, (144,288), (32,64), 64], 'stage4-4') # (14, 14, 528)
output_aux2 = auxiliary_block(x, 'aux2') # (1000, )
x = inception_block(x, [256, (160,320), (32,128), 128], 'stage4-5') # (14, 14, 832)
x = MaxPooling2D(3, 2, 'same', name='stage4_pool')(x) # (7, 7, 832)

# stage-5
x = inception_block(x, [256, (160,320), (32,128), 128], 'stage5-1') # (7, 7, 832)
x = inception_block(x, [384, (192,384), (48,128), 128], 'stage5-2') # (7, 7, 1024)
x = AveragePooling2D(7, 1, name='stage5-pool')(x) # (1, 1, 1024)

x = Flatten()(x) # (1024, )
x = Dropout(0.2)(x)
output = Dense(classes, activation='softmax', name='output')(x)

In [ ]:
model = keras.Model(
    inputs=input,
    outputs=[output , output_aux1, output_aux2]
)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stage1_conv (Conv2D)            (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
stage1_pool (MaxPooling2D)      (None, 56, 56, 64)   0           stage1_conv[0][0]                
__________________________________________________________________________________________________
stage1_bn (BatchNormalization)  (None, 56, 56, 64)   256         stage1_pool[0][0]                
______________________________________________________________________________________________

In [ ]:
keras.utils.plot_model(model, show_shapes=True)